# Pressure Levels Reanalysis: Global Timeseries

## Import libraries

Switch warnings off for better readability.

In [ ]:
import warnings

from c3s_eqc_automatic_quality_control import diagnostics, download, plot

warnings.filterwarnings("ignore")

## Define request

In [ ]:
collection_id = "reanalysis-era5-pressure-levels-monthly-means"
request = {
    "format": "grib",
    "product_type": "monthly_averaged_reanalysis",
    "variable": "temperature",
    "pressure_level": [
        "5",
        "10",
        "50",
        "100",
        "500",
        "1000",
    ],
    "time": "00:00",
}
start = "2022-01"
stop = None

## Generate requests

In [ ]:
requests = download.update_request_date(request, start=start, stop=stop)

## Compute spatially-weighted global mean timeseries

In [ ]:
ds_mean = download.download_and_transform(
    collection_id,
    requests,
    transform_func=diagnostics.spatial_weighted_mean,
    chunks={"year": 1, "month": 1},
)

## Compute spatially-weighted global std timeseries

In [ ]:
ds_std = download.download_and_transform(
    collection_id,
    requests,
    transform_func=diagnostics.spatial_weighted_std,
    chunks={"year": 1, "month": 1},
)

## Plot and save figure

In [ ]:
filename = "_".join(["global-timeseries", collection_id])
title = filename.replace("-", " ").replace("_", " ").title()
fig = plot.shaded_std("t", ds_mean, ds_std, hue_dim="plev", title=title)
fig.show()
fig.write_image(filename + ".png")